Skyline- EEG preprocessing script 
--------------------------------------------------

Steps based on the protocol:

Preliminary:
- re-referenced to average reference

Two-steps artefacts rejection:
1. ICA:
    - band pass filter using non-causal 4th order Butterworth filter between 1Hz and 30Hz
    - downsample to 256Hz
    - visual inspection to remove artefacts
    - run ICA and reject eye-movement and muscles activity artefacts
    - save ICA solution on a file
    
2. On a copy of orginal dataset:
    - filter:try out fourth order Butterworth filters: high pass causal filter at 2Hz and low-pass non-causal filter at 30Hz -> see if any distortion

Step 1: running ICA on data
-----------------------------------------

Imports

In [ ]:
import os.path as op
import mne
import numpy as np


Load data